# Chunking (*with an example using images*)

Many of the manipulations on distributed Bolt arrays `chunk` the component arrays under the hood to make operations like swapping and transposing more efficient. These chunked arrays are themselves useful for more fine-grained control over parallelization, so we expose them directly.

Let's start with a simple array

In [9]:
from bolt import ones

In [10]:
a = ones((2, 20, 10), sc)

In [3]:
a.shape

(2, 20, 10)

When we chunk, we specify the size of chunks along each axis. We only provide two sizes, not three, because we can only chunk along the axes stored in the values (which in this case includes two axes).

In [4]:
a.values.shape

(20, 10)

In [6]:
c = a.chunk((10, 5))

To see what happened, let's look at the values of the underlying RDD. Whereas initially each value was a `(20,10)` array, when chunked each one is `(10,5)`

In [6]:
a.tordd().values().first().shape

(20, 10)

In [7]:
c.tordd().values().first().shape

(10, 5)

Let's also look at the keys. They are tuples, but instead of just indexing along the first axis, they now additionally index along the chunk. Note that we've gone from 2 records in total to 8 because each subarray was broken into 4 chunks, for a total of 2 * 4.

In [8]:
a.tordd().keys().collect()

[(0,), (1,)]

In [9]:
c.tordd().keys().collect()

[((0,), (0, 0)),
 ((0,), (0, 1)),
 ((0,), (1, 0)),
 ((0,), (1, 1)),
 ((1,), (0, 0)),
 ((1,), (0, 1)),
 ((1,), (1, 0)),
 ((1,), (1, 1))]

We can perform parallelized `map` operations on the `ChunkedArray` and then `unchunk`, providing a useful interface for distributing over arrays when simply working along a subset of axes is not sufficient.

In [10]:
c.map(lambda x: x * 2).unchunk().toarray().shape

(2, 20, 10)


(2, 20, 10)

In [11]:
c.tordd().values().first()[0:2,0:2].shape

(2, 2)

In [12]:
c.tordd().keys().collect()

[((0,), (0, 0)),
 ((0,), (0, 1)),
 ((0,), (1, 0)),
 ((0,), (1, 1)),
 ((1,), (0, 0)),
 ((1,), (0, 1)),
 ((1,), (1, 0)),
 ((1,), (1, 1))]

In [21]:
from numpy import asarray

In [13]:
asarray([2]) * c.getnumber(c.plan, c.vshape)[0:1]

NameError: name 'asarray' is not defined

In [23]:
tuple(c.kshape) + tuple(asarray([4]))

(2, 4)

In [17]:
c.shape

(2, 20, 10)

In [8]:
c.map(lambda x: x[0:2,0]).unchunk().toarray().shape

[2 5]
[ 20.   1.]
(2, 20.0, 1.0)


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 7.0 failed 1 times, most recent failure: Lost task 2.0 in stage 7.0 (TID 22, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/freemanj11/code/spark-1.4.0-bin-hadoop1/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/Users/freemanj11/code/spark-1.4.0-bin-hadoop1/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/freemanj11/code/spark-1.4.0-bin-hadoop1/python/pyspark/rdd.py", line 2318, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/Users/freemanj11/code/spark-1.4.0-bin-hadoop1/python/pyspark/rdd.py", line 2318, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/Use